In [ ]:
pip install  openai langchain sentence_transformers chromadb unstructured -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.8/437.8 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.8 MB/s eta 0:00:00
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from langchain.document_loaders import DirectoryLoader

directory = '/content/drive/MyDrive/pets'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


2

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))


6


In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")



In [ ]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)


In [ ]:
query = "What are the different kinds of pets people commonly own?"
matching_docs = db.similarity_search(query)

matching_docs[0]


Document(page_content='7. **Security:** Dogs, in particular, can serve as watchdogs and provide a sense of security for their owners.\n\n8. **Learning Responsibility:** Owning a pet teaches responsibility, as owners must provide food, shelter, and care for their animals.\n\n9. **Unconditional Love:** Pets offer unwavering love and affection to their owners, creating strong emotional bonds.\n\n\n\n\n\nDisadvantages of Having Pets:**\n\n1. **Financial Costs:** Pets can be expensive to care for, with costs including food, veterinary care, grooming, and supplies.\n\n2. **Time Commitment:** Pets require time and attention. Daily feeding, exercise, and grooming routines can be demanding.\n\n3. **Allergies:** Some people are allergic to pet dander, which can cause allergic reactions, respiratory issues, and skin problems.\n\n4. **Cleanup:** Pets can be messy. They shed fur, may have accidents indoors, and require cleaning up after.', metadata={'source': '/content/drive/MyDrive/pets/pet2.txt'}

In [ ]:
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(
    documents=docs, embedding=embeddings, persist_directory=persist_directory
)

vectordb.persist()


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "key"

from langchain.chat_models import ChatOpenAI
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)


In [ ]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

query = "What are the emotional benefits of owning a pet?"
matching_docs = db.similarity_search(query)
answer =  chain.run(input_documents=matching_docs, question=query)
answer




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Benefits of Having Pets:**

1. **Companionship:** Pets provide unconditional love and companionship. They can help reduce feelings of loneliness and provide emotional support.

2. **Stress Reduction:** Interacting with pets, such as petting a cat or playing with a dog, can lower stress levels and reduce anxiety.

3. **Physical Activity:** Owning a dog encourages physical activity through daily walks and playtime, which can lead to improved fitness and a more active lifestyle.

4. **Emotional Well-being:** Pets can improve mood and boost feelings of happiness. They can also provide a sense of purpose and responsibility.

5. **Social Interaction:** Pet ownership can facilitate social interactions, as it

'The emotional benefits of owning a pet include companionship, stress reduction, improved mood, and a sense of purpose and responsibility. Pets provide unconditional love and companionship, which can help reduce feelings of loneliness and provide emotional support. Interacting with pets, such as petting a cat or playing with a dog, can lower stress levels and reduce anxiety. Pets can also improve mood and boost feelings of happiness. Additionally, owning a pet can provide a sense of purpose and responsibility, as owners must provide food, shelter, and care for their animals.'

In [ ]:
from langchain.chains import RetrievalQA
retrieval_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=db.as_retriever())
retrieval_chain.run(query)


"The emotional benefits of owning a pet include companionship, stress reduction, improved mood and happiness, a sense of purpose and responsibility, and the formation of strong emotional bonds. Pets provide unconditional love and support, which can help reduce feelings of loneliness and provide emotional comfort. Interacting with pets, such as petting a cat or playing with a dog, can also lower stress levels and reduce anxiety. Owning a pet can bring joy and happiness into one's life and provide a sense of purpose and responsibility. Additionally, the strong emotional bonds formed with pets can bring a sense of fulfillment and emotional well-being."